In [ ]:
# IP adress of device capturing images.
CAMERA_HOST = "192.168.1.106"
# Port on which the server for capturing is listening.
CAMERA_PORT = 1300

In [1]:
#!pip install Flask
import json
from flask import Flask, jsonify
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import importlib
import skimage
import json
import wandb
import socket
import json
import base64
from io import BytesIO
from PIL import Image
from IPython.display import clear_output

# Following lines were sometimes needed to be able to train on an Nvidia RTX 2060 super.
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)

#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#tf.keras.backend.set_session(tf.Session(config=config));

# Right now, this line does the trick.
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# See if GPU has been detected.
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Root directory of the project
ROOT_DIR = os.path.abspath("")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
importlib.reload(modellib)
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
class IEConfig(Config):
    """
    Configuration for running inference.
    Derives from the base Config class and overrides values specific
    to this project.
    """
    NAME = "InteractiveElements"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    CLASSES = ["Menu", "BackButton", "MoreOptions", "FloatingActionButton",
               "NavigationBarItem", "Button", "Checkbox", "RadioButton", "Switch", "Slider", "TextField"]

    # Number of classes (including background)
    NUM_CLASSES = 1 + len(CLASSES)

    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

In [ ]:
class InferenceConfig(IEConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Create the model in inference mode.
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights.
# Either set a specific path or find last trained weights.
model_path = os.path.join(ROOT_DIR, "logs/interactiveelements20221204T0029/mask_rcnn_interactiveelements_0060.h5")
# model_path = model.find_last()

# Load trained weights.
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
def getImageFromServer():
    for i in range(100):
        try:
            client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client.settimeout(1)

            client.connect((CAMERA_HOST, CAMERA_PORT))
            request = {"header": "raw"}
            client.sendall((json.dumps(request) + "\n").encode())
            image_base64 = client.makefile().readline()
            image_bytes = base64.b64decode(image_base64)
            image_file = BytesIO(image_bytes)
            image = Image.open(image_file)
            return image
        except:
            print("Connection refused:", i, "/", 100)
            continue

In [2]:
app = Flask(__name__)
@app.route('/')
def detection():
    image = getImageFromServer()
    image = np.array(image)
    
    r = model.detect([image], verbose=0)[0]
    clear_output(wait=True)
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                dataset_val.class_names, r['scores'], figsize=(16, 16))
    return jsonify(r)

app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2022 10:14:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2022 10:14:52] "HEAD / HTTP/1.1" 200 -
